In [ ]:
from skimage.io import imread, imshow, imread_collection, concatenate_images
import matplotlib.pyplot as plt
import os
import sys
import random
import warnings

import numpy as np
from numpy import fliplr, flipud
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from tensorflow import keras
from keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, BatchNormalization
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf


In [ ]:
DIRin1 = "../input/severstal-steel-defect-detection"
DIRtrain = os.path.join(DIRin1,"train_images")
DIRtest = os.path.join(DIRin1,"test_images")
train = pd.read_csv("/kaggle/input/severstal-steel-defect-detection/train.csv")
IMAGE_PATH = r"/kaggle/input/severstal-steel-defect-detection/train_images/"
TEST_IMAGE_PATH = r"/kaggle/input/severstal-steel-defect-detection/test_images/"

In [ ]:
# import pandas_profiling as pp
# pp.ProfileReport(train)

In [ ]:
train["ClassId"].value_counts().plot(kind = 'bar')
train["ClassId"].value_counts()

In [ ]:
class_indices=[[],[],[],[]]
for ind in range(train.shape[0]):
    class_indices[train['ClassId'][ind]-1].append(ind)

In [ ]:
sample_set = []    
    
for defect_type in range(4):
    partial_sample_set = np.random.permutation(240)
    for sample_index in partial_sample_set:
        sample_set.append(class_indices[defect_type][sample_index])


In [ ]:
def rle2mask(mask_rle, shape=(1600,256)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
img = rle2mask(train['EncodedPixels'][1])
plt.imshow(img)

In [ ]:
Y_train = np.zeros((3840,128, 800,4 ), dtype=bool)
n=0        
for ind in range(len(sample_set)):
    img = rle2mask(train['EncodedPixels'][sample_set[ind]])
    img = resize(img, (128, 800), mode='constant', preserve_range=True)
    Y_train[n,:,:,(train['ClassId'][sample_set[ind]]-1)] = img
    Y_train[n+1,:,:,(train['ClassId'][sample_set[ind]]-1)] = fliplr(img)
    Y_train[n+2,:,:,(train['ClassId'][sample_set[ind]]-1)] = flipud(img)
    Y_train[n+3,:,:,(train['ClassId'][sample_set[ind]]-1)] = fliplr(flipud(img))
    n+=4

In [ ]:
X_train = np.zeros((3840, 128, 800, 3), dtype=np.uint8)
n = 0
for ind in range(len(sample_set)):
    img = imread(IMAGE_PATH + train['ImageId'][sample_set[ind]])
    img = resize(img, (128, 800, 3), mode='constant', preserve_range=True)
    X_train[n,:,:,:] = img
    X_train[n+1,:,:,:] = fliplr(img)
    X_train[n+2,:,:,:] = flipud(img)
    X_train[n+3,:,:,:] = fliplr(flipud(img))
    n += 4

In [ ]:

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
inputs = Input((128, 800, 3))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.2) (c1)
c1 = BatchNormalization()(c1)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
c1 = BatchNormalization()(c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.3) (c2)
c2 = BatchNormalization()(c2)
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
c2 = BatchNormalization()(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.3) (c3)
c3 = BatchNormalization()(c3)
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
c3 = BatchNormalization()(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.4) (c4)
c4 = BatchNormalization()(c4)
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
c4 = BatchNormalization()(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.5) (c5)
c5 = BatchNormalization()(c5)
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
u6 = BatchNormalization()(u6)
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.4) (c6)
c6 = BatchNormalization()(c6)
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)
c6 = BatchNormalization()(c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
u7 = BatchNormalization()(u7)
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.3) (c7)
c7 = BatchNormalization()(c7)
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
u8 = BatchNormalization()(u8)
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.3) (c8)
c8 = BatchNormalization()(c8)
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)
c8 = BatchNormalization()(c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
u9 = BatchNormalization()(u9)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.2) (c9)
c9 = BatchNormalization()(c9)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)
c9 = BatchNormalization()(c9)

outputs = Conv2D(4, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
adamcustom = Adam(lr=0.0003)
model.compile(optimizer=adamcustom , loss=['binary_crossentropy'], metrics=[dice_coef])
model.summary()

In [ ]:
results = model.fit(X_train, Y_train, validation_split=0.1, shuffle=True, batch_size=16, epochs=50)

In [ ]:
plt.figure(figsize=(18, 6))
plt.plot( results.history['loss'], label="train loss")
plt.plot( results.history['val_loss'], label="valid loss")
plt.show()

In [ ]:
plt.figure(figsize=(18, 6))
plt.plot( results.history['dice_coef'], label="train loss")
plt.plot( results.history['val_dice_coef'], label="valid loss")
plt.show()

In [ ]:
print("max dice_coef :",np.amax(results.history['dice_coef']))
print("max val_dice_coef :",np.amax(results.history['val_dice_coef']))

In [ ]:
print("mean dice_coef :",np.mean(results.history['dice_coef']))
print("mean val_dice_coef :",np.mean(results.history['val_dice_coef']))

In [ ]:
model.save("steel_unet.model", save_format="h5")

In [ ]:
model2 = load_model("../input/model2/steel_unet.model",compile=False)

In [ ]:
img1 = rle2mask(train['EncodedPixels'][0])
img2 = resize(img1, (128, 800), mode='constant', preserve_range=True)
plt.imshow(img2)

In [ ]:
X_test = np.zeros((1,128,800,3), dtype=np.uint8)
img = cv2.imread(TEST_IMAGE_PATH+'0000f269f.jpg')
plt.imshow(img)

In [ ]:
img = resize(img, (128,800,3), mode='constant', preserve_range=True)
X_test[0,:,:,:] = img
pred_masks = model2.predict(X_test[0:1,:,:,:])
preds_test_t = (pred_masks > 0.5).astype(np.uint8)

In [ ]:
plt.imshow(preds_test_t[0,:,:,i])

In [ ]:
# X_test = np.zeros((len(testfiles), 128,800,3), dtype=np.uint8)

# n = 0
# for file in testfiles:
#     img = imread(TEST_IMAGE_PATH + file)
#     img = resize(img, (128,800,3), mode='constant', preserve_range=True)
#     X_test[n,:,:,:] = img
#     n += 1
#     if n % 1000 == 0:
#         print('Saving entry', n)

In [ ]:
# SAVE MODEL
# model.load_weights('model.h5')
#model.save('UNET.h5')


# # LOAD MODEL
# from keras.models import load_model
# model1 = load_model('UNET.h5',custom_objects={'dice_coef':dice_coef})

In [ ]:
temp_df = pd.DataFrame()
x= pd.read_csv(os.path.join(DIRin1,'sample_submission.csv'))
testfiles = x['ImageId']

In [ ]:
def mask2rle(img):
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
submit_df= pd.read_csv('../input/sample-sub/new_sample_submission.csv')
print(submit_df)

In [ ]:
X_test = np.zeros((1,128,800,3), dtype=np.uint8)

n=0
for i in tqdm(testfiles): 
    img = cv2.imread(os.path.join(DIRtest,i))
    img = resize(img, (128,800,3), mode='constant', preserve_range=True)
    X_test[0,:,:,:] = img
    pred_masks = model.predict(X_test[0:1,:,:,:])
    preds_test_t = (pred_masks > 0.5).astype(np.uint8)
    for m in range(4):
        img = preds_test_t[0,:,:,m]
        img = resize(img, (256, 1600), mode='constant', preserve_range=True)
        encoded_entry = mask2rle(img)
        if len(encoded_entry)>0:
            submit_df.iloc[n*4+m,1] = encoded_entry
    n+=1

#rles = build_rles(pred_masks)

In [ ]:
submit_df.to_csv('./submission.csv', index=False)

In [ ]:
X_test1= np.zeros((1,128,800,3), dtype=np.uint8)
print(testfiles[0])
img= imread(TEST_IMAGE_PATH + testfiles[108])
img = resize(img, (128,800,3))
plt.imshow(img)
X_test1[0,:,:,:] = img
pred_masks1 = model.predict(X_test1[0:1,:,:,:])
preds_test_t1 = (pred_masks1 > 0.5).astype(np.uint8)

In [ ]:
# encoded_entry = mask2rle(preds_test_t1[0,:,:,0])
# print(encoded_entry)

In [ ]:
plt.imshow( preds_test_t1[0,:,:,0])

In [ ]:
 plt.imshow( preds_test_t1[0,:,:,1])

In [ ]:
 plt.imshow( preds_test_t1[0,:,:,2])

In [ ]:
plt.imshow( preds_test_t1[0,:,:,3])

In [ ]:
#predict results
#model = load_model('UNET.h5')

# submission_list = []
# for n in range(len(testfiles)):
#     preds_test = model.predict(X_test[n:n+1,:,:,:])
#     preds_test_t = (preds_test > 0.5).astype(np.uint8)
#     for m in range(4):
#         img = preds_test_t[0,:,:,m]
#         img = resize(img, (256, 1600), mode='constant', preserve_range=True)
#         encoded_entry = mask2rle(img)
#         row = [testfiles[n] + '_' + str(m+1) , encoded_entry]
#         submission_list.append(row)
#     if n % 1000 == 0:
#         print('Saving entry', n)
    
# submission_data = pd.DataFrame(submission_list, columns=['ImageId_ClassId','EncodedPixels'])
# submission_data.to_csv('./submission.csv', index=False)
                

In [ ]:
# submit_df = pd.DataFrame(columns=['ImageId_ClassId','EncodedPixels'])
# for i in tqdm(testfiles):
#     for j in range(4):
#         tmp_se = pd.Series( [i+'_{}'.format(j+1),None], index=submit_df.columns )
#         submit_df = submit_df.append( tmp_se, ignore_index=True )
        

In [ ]:
print(submit_df)